In [ ]:
# =============================================================================
# BLOCO 2A: VERIFICAÇÃO DO SINAL DE FALHA PURO (DOMÍNIO DO TEMPO)
# =============================================================================
import matplotlib.pyplot as plt
import numpy as np

print("--- PLOTANDO SINAL DE FALHA PURO (SINTÉTICO, ANTES DA SOMA) ---")

# --- 1. CONFIGURAÇÕES ---
classes_para_plotar = ['Pista Externa', 'Pista Interna', 'Esfera']
ZOOM_TIME_LIMIT_SEC = 0.2 # Zoom nos primeiros 200ms para ver a modulação
# Vamos pegar o multiplicador mais forte (100x) para ver o sinal de falha claramente
MULTIPLICADOR_ALVO = 5 

# Cria 3 subplots (um para cada falha)
fig, ax = plt.subplots(len(classes_para_plotar), 1, figsize=(15, 15))
fig.suptitle(f'Sinal de Falha Puro (Mult: {MULTIPLICADOR_ALVO}x) - Zoom 0-{ZOOM_TIME_LIMIT_SEC}s', fontsize=16)

# --- 2. LOOP POR CADA TIPO DE FALHA ---
try:
    for i, tipo_falha in enumerate(classes_para_plotar):
        
        ax_atual = ax[i]
        
        # --- 3. ENCONTRAR O SINAL SINTÉTICO (Combinado) ---
        # (Usa o df_sinais_treino que está em memória do Bloco 2)
        query_str = (f"tipo_falha_adicionada == '{tipo_falha}' and "
                     f"multiplicador_amplitude == {MULTIPLICADOR_ALVO}")
        query_sintetico = df_sinais_treino.query(query_str)
        
        if query_sintetico.empty:
            ax_atual.set_title(f"Sinal Sintético '{tipo_falha}' (Mult: {MULTIPLICADOR_ALVO}x) NÃO ENCONTRADO")
            continue
        
        amostra = query_sintetico.iloc[0]
        sinal_final_combinado = amostra['sinal_final']
        rpm_sint = amostra['rpm']
        chave_base_normal = amostra['base_normal']
        
        # --- 4. ENCONTRAR O SINAL NORMAL ORIGINAL (Base) ---
        # (Usa o dicionario_treino que está em memória do Bloco 1)
        if chave_base_normal not in dicionario_treino:
            ax_atual.set_title(f"Base Normal '{chave_base_normal}' NÃO ENCONTRADA NA MEMÓRIA")
            continue
        
        sinal_normal_base = dicionario_treino[chave_base_normal]['amplitude'].values
        
        # --- 5. SUBTRAIR A BASE PARA OBTER O SINAL "PURO" ---
        n_pontos = min(len(sinal_final_combinado), len(sinal_normal_base))
        sinal_falha_puro = sinal_final_combinado[:n_pontos] - sinal_normal_base[:n_pontos]
        
        # --- 6. CALCULAR TEMPO E PERÍODO ---
        t_sint = np.linspace(0, n_pontos / TAXA_AMOSTRAL, n_pontos, endpoint=False)
        
        fr_hz_sint = rpm_sint / 60
        # (A função 'calcular_frequencias_rolamento' DEVE estar em memória do Bloco 2)
        freqs_teoricas = calcular_frequencias_rolamento(fr=fr_hz_sint, **params_drive_end)
        freq_teorica_falha = freqs_teoricas[tipo_falha]
        periodo_falha_sec = 1.0 / freq_teorica_falha
        
        # --- 7. PLOTAR O SINAL DE FALHA PURO ---
        ax_atual.plot(t_sint, sinal_falha_puro, color='red', label='Sinal de Falha Puro')
        ax_atual.set_title(f"SINAL PURO: {tipo_falha} @ {rpm_sint} RPM (Mult: {MULTIPLICADOR_ALVO}x)")
        ax_atual.set_xlim(0, ZOOM_TIME_LIMIT_SEC)
        ax_atual.set_ylabel("Amplitude da Falha")
        ax_atual.grid(True, linestyle='--', alpha=0.6)
        
        # Adicionar marcadores de período de falha (quando o impacto ocorre)
        for t_impacto in np.arange(0, ZOOM_TIME_LIMIT_SEC, periodo_falha_sec):
            ax_atual.axvline(x=t_impacto, color='black', linestyle=':', alpha=0.8, 
                             label=f'Período ({periodo_falha_sec*1000:.1f} ms)' if t_impacto < 1e-6 else None)
        ax_atual.legend(loc='upper right')

    # Adicionar xlabel no último gráfico
    ax[len(classes_para_plotar)-1].set_xlabel('Tempo (s)')
    
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

except NameError as e:
    print(f"\nERRO DE EXECUÇÃO: {e}")
    print("Certifique-se de que o Bloco 1 (para 'dicionario_treino') e o Bloco 2 (para 'df_sinais_treino' e 'calcular_frequencias_rolamento')")
    print("foram executados nesta sessão antes de rodar este bloco de plotagem.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado no plot: {e}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

print("--- COMPARAÇÃO NO DOMÍNIO DO TEMPO (Real vs. Novo Modelo Sintético) ---")

# --- 1. CONFIGURAÇÕES ---
classes_para_plotar = ['Normal', 'Pista Externa', 'Pista Interna', 'Esfera']
ZOOM_TIME_LIMIT_SEC = 0.1 # Zoom nos primeiros 100ms para ver os pulsos

# Criar a grade de plots
fig, ax = plt.subplots(len(classes_para_plotar), 2, figsize=(20, 18))
fig.suptitle(f'Comparação no Tempo: Real (Esquerda) vs. Novo Sintético (Direita) - Zoom 0-{ZOOM_TIME_LIMIT_SEC}s', fontsize=18)

# --- 2. FUNÇÕES AUXILIARES (copiadas para a célula ser independente) ---
def encontrar_primeiro_segmento_real(tipo_falha):
    dicionario_fonte = dicionario_treino if tipo_falha == 'Normal' else dicionario_teste
    for df_seg in dicionario_fonte.values():
        if df_seg['tipo_falha'].iloc[0] == tipo_falha:
            return df_seg['amplitude'].values, df_seg['rotacao_rpm'].iloc[0]
    return None, None

def encontrar_primeiro_segmento_sintetico(tipo_falha):
    query_str = f"tipo_falha_adicionada == '{tipo_falha}'"
    query_sintetico = df_sinais_treino.query(query_str)
    if not query_sintetico.empty:
        amostra = query_sintetico.iloc[0]
        return amostra['sinal_final'], amostra['rpm']
    return None, None

# --- 3. LOOP DE PLOTAGEM ---
try:
    for i, tipo_falha in enumerate(classes_para_plotar):
        
        ax_real = ax[i, 0]
        ax_sint = ax[i, 1]

        # --- A. SINAL REAL (Esquerda) ---
        sinal_real, rpm_real = encontrar_primeiro_segmento_real(tipo_falha)
        freq_teorica = None
        
        if sinal_real is not None:
            N_real = len(sinal_real)
            t_real = np.linspace(0, N_real / TAXA_AMOSTRAL, N_real, endpoint=False)
            
            if tipo_falha != 'Normal':
                fr_hz_real = rpm_real / 60
                freq_teorica = calcular_frequencias_rolamento(fr=fr_hz_real, **params_drive_end)[tipo_falha]
                periodo_falha_sec = 1.0 / freq_teorica

            # Plot no tempo
            ax_real.plot(t_real, sinal_real, color='blue', label='Sinal Real')
            ax_real.set_title(f"SINAL REAL: {tipo_falha} (RPM: {rpm_real})")
            ax_real.set_xlim(0, ZOOM_TIME_LIMIT_SEC) # Aplica o Zoom
            ax_real.set_ylabel("Amplitude")
            ax_real.grid(True, linestyle='--', alpha=0.6)
            
            if freq_teorica:
                # Adiciona marcadores de período de falha
                for t_impacto in np.arange(periodo_falha_sec, ZOOM_TIME_LIMIT_SEC, periodo_falha_sec):
                    ax_real.axvline(x=t_impacto, color='red', linestyle='--', alpha=0.8, 
                                    label=f'Período ({periodo_falha_sec*1000:.1f} ms)' if t_impacto==periodo_falha_sec else None)
                ax_real.legend(loc='upper right')
        else:
            ax_real.set_title(f"SINAL REAL '{tipo_falha}' NÃO ENCONTRADO")

        # --- B. SINAL SINTÉTICO (Direita) ---
        sinal_sint, rpm_sint = encontrar_primeiro_segmento_sintetico(tipo_falha)
        
        if sinal_sint is not None:
            N_sint = len(sinal_sint)
            t_sint = np.linspace(0, N_sint / TAXA_AMOSTRAL, N_sint, endpoint=False)
            freq_teorica_sint = None
            
            if tipo_falha != 'Normal':
                fr_hz_sint = rpm_sint / 60
                freq_teorica_sint = calcular_frequencias_rolamento(fr=fr_hz_sint, **params_drive_end)[tipo_falha]
                periodo_falha_sec_sint = 1.0 / freq_teorica_sint

            # Plot no tempo
            ax_sint.plot(t_sint, sinal_sint, color='green', label='Sinal Sintético (White)')
            ax_sint.set_title(f"SINTÉTICO (Eq. 9): {tipo_falha} (RPM: {rpm_sint})")
            ax_sint.set_xlim(0, ZOOM_TIME_LIMIT_SEC) # Aplica o Zoom
            ax_sint.grid(True, linestyle='--', alpha=0.6)

            if freq_teorica_sint:
                # Adiciona marcadores de período de falha
                for t_impacto_sint in np.arange(periodo_falha_sec_sint, ZOOM_TIME_LIMIT_SEC, periodo_falha_sec_sint):
                    ax_sint.axvline(x=t_impacto_sint, color='red', linestyle='--', alpha=0.8, 
                                     label=f'Período ({periodo_falha_sec_sint*1000:.1f} ms)' if t_impacto_sint==periodo_falha_sec_sint else None)
                ax_sint.legend(loc='upper right')
        else:
            ax_sint.set_title(f"SINTÉTICO '{tipo_falha}' NÃO ENCONTRADO")

        # Adiciona xlabel apenas na última linha
        if i == len(classes_para_plotar) - 1:
            ax_real.set_xlabel('Tempo (s)')
            ax_sint.set_xlabel('Tempo (s)')

    fig.tight_layout(rect=[0, 0.03, 1, 0.96])
    plt.show()

except NameError as e:
    print(f"\nERRO: Variável não encontrada ({e}).")
    print("Certifique-se de ter executado o Bloco 1 e o Bloco 2 (o novo) primeiro.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado: {e}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# --- 1. CONFIGURAÇÕES DO PLOT ---
# =====================================================================
# *** Mude aqui para o RPM que você quer inspecionar ***
RPM_PARA_PLOTAR = 1750  # Ex: 1730, 1750, 1773, ou 1797
# =====================================================================
MULTIPLICADOR_PARA_PLOTAR = 10 # Você pediu 10x

classes_para_plotar = ['Normal', 'Pista Externa', 'Pista Interna', 'Esfera']
XLIM_ZOOM = 800  # 0-800 Hz (Foco na freq. de falha e harmônicos)

print(f"--- COMPARANDO FFT @ {RPM_PARA_PLOTAR} RPM (Sintético com Mult: {MULTIPLICADOR_PARA_PLOTAR}x) ---")

# Cria a grade de plots
fig, ax = plt.subplots(len(classes_para_plotar), 2, figsize=(20, 18))
fig.suptitle(f'FFT Real vs. Sintético @ {RPM_PARA_PLOTAR} RPM (Mult: {MULTIPLICADOR_PARA_PLOTAR}x)', fontsize=18)

# --- 2. FUNÇÕES AUXILIARES DE BUSCA (Mais específicas) ---
def encontrar_segmento_real_especifico(tipo_falha, rpm_desejado):
    """Encontra o primeiro segmento REAL com um tipo de falha e RPM específicos."""
    dicionario_fonte = dicionario_treino if tipo_falha == 'Normal' else dicionario_teste
    
    for df_seg in dicionario_fonte.values():
        if (df_seg['tipo_falha'].iloc[0] == tipo_falha and 
            df_seg['rotacao_rpm'].iloc[0] == rpm_desejado):
            return df_seg['amplitude'].values, df_seg['rotacao_rpm'].iloc[0]
    return None, None # Não encontrou

def encontrar_segmento_sintetico_especifico(tipo_falha, rpm_desejado, multiplicador):
    """Encontra o primeiro segmento SINTÉTICO com filtros específicos."""
    if tipo_falha == 'Normal':
        # 'Normal' não tem multiplicador, busca só por RPM
        query_str = f"tipo_falha_adicionada == 'Normal' and rpm == {rpm_desejado}"
    else:
        # Falhas buscam por RPM e Multiplicador
        query_str = (f"tipo_falha_adicionada == '{tipo_falha}' and "
                     f"rpm == {rpm_desejado} and "
                     f"multiplicador_amplitude == {multiplicador}")
    
    query_sintetico = df_sinais_treino.query(query_str)
    if not query_sintetico.empty:
        amostra = query_sintetico.iloc[0]
        return amostra['sinal_final'], amostra['rpm']
    return None, None # Não encontrou

# --- 3. LOOP DE PLOTAGEM ---
try:
    for i, tipo_falha in enumerate(classes_para_plotar):
        
        ax_real = ax[i, 0]
        ax_sint = ax[i, 1]

        # --- A. SINAL REAL (Esquerda) ---
        sinal_real, rpm_real = encontrar_segmento_real_especifico(tipo_falha, RPM_PARA_PLOTAR)
        freq_teorica = None
        
        if sinal_real is not None:
            # Calcula FFT Real
            N_real = len(sinal_real)
            espectro_real = np.abs(np.fft.fft(sinal_real)[0:N_real//2]) / (N_real/2)
            freqs_real = np.fft.fftfreq(N_real, 1 / TAXA_AMOSTRAL)[:N_real//2]
            
            # Calcula Freq. Teórica (se não for 'Normal')
            if tipo_falha != 'Normal':
                fr_hz_real = rpm_real / 60
                freq_teorica = calcular_frequencias_rolamento(fr=fr_hz_real, **params_drive_end)[tipo_falha]

            # Plota Real (Zoom)
            ax_real.plot(freqs_real, espectro_real, color='blue')
            ax_real.set_title(f'SINAL REAL: {tipo_falha} @ {rpm_real} RPM')
            ax_real.set_ylabel('Amplitude')
            ax_real.set_xlim(0, XLIM_ZOOM)
            ax_real.grid(True, linestyle='--', alpha=0.6)
            
            if freq_teorica:
                ax_real.axvline(x=freq_teorica, color='red', linestyle='--', label=f'Teórica: {freq_teorica:.2f} Hz')
                for h in range(2, 6): ax_real.axvline(x=freq_teorica * h, color='orange', linestyle=':', alpha=0.7)
                ax_real.legend()
        else:
            ax_real.set_title(f"SINAL REAL '{tipo_falha}' @ {RPM_PARA_PLOTAR} RPM\nNÃO ENCONTRADO")

        # --- B. SINAL SINTÉTICO (Direita) ---
        sinal_sint, rpm_sint = encontrar_segmento_sintetico_especifico(tipo_falha, RPM_PARA_PLOTAR, MULTIPLICADOR_PARA_PLOTAR)
        
        if sinal_sint is not None:
            # Calcula FFT Sintético
            N_sint = len(sinal_sint)
            espectro_sint = np.abs(np.fft.fft(sinal_sint)[0:N_sint//2]) / (N_sint/2)
            freqs_sint = np.fft.fftfreq(N_sint, 1 / TAXA_AMOSTRAL)[:N_sint//2]

            # Plota Sintético (Zoom)
            ax_sint.plot(freqs_sint, espectro_sint, color='green')
            ax_sint.set_title(f'SINTÉTICO ({MULTIPLICADOR_PARA_PLOTAR}x): {tipo_falha} @ {rpm_sint} RPM')
            ax_sint.set_xlim(0, XLIM_ZOOM)
            ax_sint.grid(True, linestyle='--', alpha=0.6)
            
            if freq_teorica: # Usa a mesma freq teórica do real para consistência
                ax_sint.axvline(x=freq_teorica, color='red', linestyle='--', label=f'Teórica: {freq_teorica:.2f} Hz')
                for h in range(2, 6): ax_sint.axvline(x=freq_teorica * h, color='orange', linestyle=':', alpha=0.7)
                ax_sint.legend()
        else:
            ax_sint.set_title(f"SINTÉTICO ({MULTIPLICADOR_PARA_PLOTAR}x) '{tipo_falha}' @ {RPM_PARA_PLOTAR} RPM\nNÃO ENCONTRADO")

        # Adiciona xlabel apenas na última linha
        if i == len(classes_para_plotar) - 1:
            ax_real.set_xlabel('Frequência (Hz)')
            ax_sint.set_xlabel('Frequência (Hz)')

    fig.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajuste para o supertítulo
    plt.show()

except NameError as e:
    print(f"\nERRO: Variável não encontrada ({e}).")
    print("Certifique-se de ter executado o Bloco 1 e o Bloco 2 (com o novo modelo) primeiro.")
except Exception as e:
    print(f"\nOcorreu um erro inesperado: {e}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

print("--- ENCONTRANDO O PICO DE RESSONÂNCIA REAL ---")

try:
    # --- 1. ESCOLHA UM SINAL DE FALHA REAL PARA MEDIR ---
    # (Vamos usar 'Pista Externa' como nosso medidor)
    sinal_real, rpm_real = None, None
    for df_seg in dicionario_teste.values():
        if df_seg['tipo_falha'].iloc[0] == 'Pista Externa':
            sinal_real = df_seg['amplitude'].values
            rpm_real = df_seg['rotacao_rpm'].iloc[0]
            break
            
    if sinal_real is None:
        raise Exception("Nenhum sinal real de 'Pista Externa' encontrado no dicionario_teste para medir.")

    # --- 2. CALCULAR O FFT REAL ---
    N_real = len(sinal_real)
    espectro_real = np.abs(np.fft.fft(sinal_real)[0:N_real//2]) / (N_real/2)
    freqs_real = np.fft.fftfreq(N_real, 1 / TAXA_AMOSTRAL)[:N_real//2]
    
    # --- 3. ENCONTRAR O PICO DE RESSONÂNCIA (O PONTO CRÍTICO) ---
    
    # Ignora as frequências de falha de baixa frequência (ex: < 1000 Hz)
    limite_baixa_freq_hz = 1000 
    
    # Cria uma máscara para focar apenas na região de ressonância
    mascara_alta_freq = freqs_real > limite_baixa_freq_hz
    
    # Encontra o índice do pico (amplitude máxima) *apenas* na região de alta frequência
    indice_pico_hf = np.argmax(espectro_real[mascara_alta_freq])
    
    # Converte esse índice de volta para um índice do array original
    indice_pico_global = np.where(mascara_alta_freq)[0][indice_pico_hf]
    
    # Obtém a frequência exata desse pico
    freq_pico_real = freqs_real[indice_pico_global]
    amplitude_pico_real = espectro_real[indice_pico_global]

    print("\n" + "="*50)
    print(f"  PICO DE RESSONÂNCIA REAL ENCONTRADO!")
    print(f"  Frequência: {freq_pico_real:.2f} Hz")
    print(f"  Amplitude: {amplitude_pico_real:.4f}")
    print("="*50 + "\n")
    print(f"Instrução: Use '{freq_pico_real:.2f}' como o novo valor para 'freq_natural_hz' no Bloco 2.")

    # --- 4. PLOTAR PARA VERIFICAÇÃO VISUAL ---
    plt.figure(figsize=(15, 6))
    plt.plot(freqs_real, espectro_real, label='Sinal Real (Pista Externa)')
    
    # Desenha a linha vertical no pico que encontramos
    plt.axvline(x=freq_pico_real, color='red', linestyle='--', 
                label=f'Pico de Ressonância Encontrado: {freq_pico_real:.2f} Hz')
    
    plt.title('Identificação da Frequência Natural (Ressonância) do Sinal Real', fontsize=16)
    plt.xlabel('Frequência (Hz)')
    plt.ylabel('Amplitude Normalizada')
    plt.xlim(0, TAXA_AMOSTRAL / 2) # Espectro completo
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

except NameError as e:
    print(f"\nERRO: Variável não encontrada ({e}). Certifique-se de que o Bloco 1 foi executado.")
except Exception as e:
    print(f"\nOcorreu um erro: {e}")